In [1]:
from enum import Enum, IntEnum, auto

## Enumerations for entities in the puzzle

class House(Enum):
    RED = auto()
    GREEN = auto()
    WHITE = auto()
    YELLOW = auto()
    BLUE = auto()

class Person(Enum):
    BRIT = auto()
    SWEDE = auto()
    DANE = auto()
    NORWEGIAN = auto()
    GERMAN = auto()

class Beverage(Enum):
    TEA = auto()
    COFFEE = auto()
    MILK = auto()
    BEER = auto()
    WATER = auto()

class Cigar(Enum):
    PALL_MALL = auto()
    DUNHILL = auto()
    BLENDS = auto()
    BLUE_MASTER = auto()
    PRINCE = auto()

class Pet(Enum):
    DOGS = auto()
    BIRDS = auto()
    CATS = auto()
    HORSES = auto()
    FISH = auto()

class Location(IntEnum):
    FIRST = auto()
    SECOND = auto()
    THIRD = auto()
    FOURTH = auto()
    FIFTH = auto()

In [2]:
import weakref
from dataclasses import dataclass

## Meta class for propositions

class PropType(type):
    __count = 0
    __instances = weakref.WeakValueDictionary()
    __id_map = {}
    __init_stage = True

    def __call__(self, *args, **kwargs):
        signature = (self, *args, *kwargs.values())
        if signature in PropType.__instances:
            return PropType.__instances[signature]
        else:
            assert PropType.__init_stage, "Unexpected new proposition after init stage"
            PropType.__count += 1
            instance = super().__call__(*args, **kwargs)
            instance.id = PropType.__count
            PropType.__instances[signature] = instance
            PropType.__id_map[PropType.__count] = signature
            return instance

    @classmethod
    def freeze(cls):
        PropType.__init_stage = False

    @classmethod
    def from_id(cls, id):
        return PropType.__instances.get(PropType.__id_map[id])


def PropClass(cls):
    dcls = dataclass(cls, frozen=True)
    class Decorated(dcls, metaclass=PropType):
        __hash__ = dcls.__hash__

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
        
        def __int__(self):
            return self.id

        def __eq__(self, other):
            return type(self) is type(other) and self.id == other.id

    Decorated.__name__ = cls.__name__
    Decorated.__qualname__ = cls.__qualname__
    return Decorated

In [3]:
## Propositions

@PropClass
class Lives():
    person: Person
    house: House


@PropClass
class Drinks():
    person: Person
    beverage: Beverage


@PropClass
class Smokes():
    person: Person
    cigar: Cigar


@PropClass
class HasPet():
    person: Person
    pet: Pet


@PropClass
class LocatesAt():
    house: House
    location: Location


props_lives = [Lives(p, h) for p in Person for h in House]
props_drinks = [Drinks(p, b) for p in Person for b in Beverage]
props_smokes = [Smokes(p, c) for p in Person for c in Cigar]
props_has_pet = [HasPet(p, pet) for p in Person for pet in Pet]
props_locates_at = [LocatesAt(h, l) for h in House for l in Location]
props = props_lives + props_drinks + props_smokes + props_has_pet + props_locates_at

PropType.freeze()

for prop in props:
    print(f'Prop #{int(prop)}:\t{prop}')

Prop #1:	Lives(person=<Person.BRIT: 1>, house=<House.RED: 1>)
Prop #2:	Lives(person=<Person.BRIT: 1>, house=<House.GREEN: 2>)
Prop #3:	Lives(person=<Person.BRIT: 1>, house=<House.WHITE: 3>)
Prop #4:	Lives(person=<Person.BRIT: 1>, house=<House.YELLOW: 4>)
Prop #5:	Lives(person=<Person.BRIT: 1>, house=<House.BLUE: 5>)
Prop #6:	Lives(person=<Person.SWEDE: 2>, house=<House.RED: 1>)
Prop #7:	Lives(person=<Person.SWEDE: 2>, house=<House.GREEN: 2>)
Prop #8:	Lives(person=<Person.SWEDE: 2>, house=<House.WHITE: 3>)
Prop #9:	Lives(person=<Person.SWEDE: 2>, house=<House.YELLOW: 4>)
Prop #10:	Lives(person=<Person.SWEDE: 2>, house=<House.BLUE: 5>)
Prop #11:	Lives(person=<Person.DANE: 3>, house=<House.RED: 1>)
Prop #12:	Lives(person=<Person.DANE: 3>, house=<House.GREEN: 2>)
Prop #13:	Lives(person=<Person.DANE: 3>, house=<House.WHITE: 3>)
Prop #14:	Lives(person=<Person.DANE: 3>, house=<House.YELLOW: 4>)
Prop #15:	Lives(person=<Person.DANE: 3>, house=<House.BLUE: 5>)
Prop #16:	Lives(person=<Person.NORW

In [4]:
## CNF class definition

class Not():
    def __init__(self, prop):
        self.prop = prop

    def __str__(self):
        return f'~{self.prop}'

    def __repr__(self):
        return f'Not({repr(self.prop)})'

    def __eq__(self, other):
        return isinstance(other, Not) and self.prop == other.prop

    def __hash__(self):
        return hash(self.prop)

    def __int__(self):
        return -int(self.prop)


class Clause():
    def __init__(self, *props):
        self.props = props

    def __str__(self):
        return f'({", ".join(str(prop) for prop in self.props)})'

    def __repr__(self):
        return f'Clause({", ".join(repr(prop) for prop in self.props)})'

    def __iter__(self):
        return map(int, self.props)

    def verify(self, model):
        return any(int(prop) in model for prop in self.props)


class CNF():
    def __init__(self, *clauses):
        self.clauses = clauses

    def __str__(self):
        return f'[{", ".join(str(clause) for clause in self.clauses)}]'

    def __repr__(self):
        return f'CNF[{", ".join(repr(clause) for clause in self.clauses)}]'

    def __iter__(self):
        return map(list, self.clauses)

    def __len__(self):
        return len(self.clauses)

    def __getitem__(self, index):
        return self.clauses[index]

    def append(self, *clauses):
        self.clauses += clauses

    def verify(self, model):
        for clause in self.clauses:
            if not clause.verify(model):
                return False
        return True

In [5]:
## Building the CNF
## Syntax of each clause:
## ~(p1 & p2 & ... & pn) == (~p1 | ~p2 | ... | ~pn)
## ((p1 & p2 & ... & pn) -> (q1 | q2 | ... | qm)) == (~p1 | ~p2 | ... | ~pn | q1 | q2 | ... | qm)
## (p -> ((q1 -> r1) & (q2 -> r2) & ... & (qm -> rm))) == (~p | ~q1 | r1) & (~p | ~q2 | r2) & ... & (~p | ~qm | rm)

cnf = CNF()

## -- Constraints --
# C-1. There are five houses in five different colors.
cnf.append(*[
    Clause(*[LocatesAt(h, l) for l in Location])
    for h in House])
cnf.append(*[
    Clause(Not(LocatesAt(h, l1)), Not(LocatesAt(h, l2)))
    for h in House for l1 in Location for l2 in Location
    if l1 != l2])
cnf.append(*[
    Clause(Not(LocatesAt(h1, l)), Not(LocatesAt(h2, l)))
    for h1 in House for h2 in House for l in Location
    if h1 != h2])

# C-2. In each house lives a man with different nationality.
cnf.append(*[
    Clause(*[Lives(p, h) for p in Person])
    for h in House])
cnf.append(*[
    Clause(Not(Lives(p1, h)), Not(Lives(p2, h)))
    for p1 in Person for p2 in Person for h in House
    if p1 != p2])
cnf.append(*[
    Clause(Not(Lives(p, h1)), Not(Lives(p, h2)))
    for p in Person for h1 in House for h2 in House
    if h1 != h2])

# C-3. These five owners drink a certain type of beverage, smoke a certain brand of cigar and keep a certain pet.
cnf.append(*[
    Clause(*[Drinks(p, b) for b in Beverage])
    for p in Person])
cnf.append(*[
    Clause(Not(Drinks(p, b1)), Not(Drinks(p, b2)))
    for p in Person for b1 in Beverage for b2 in Beverage
    if b1 != b2])
cnf.append(*[
    Clause(*[Smokes(p, c) for c in Cigar])
    for p in Person])
cnf.append(*[
    Clause(Not(Smokes(p, c1)), Not(Smokes(p, c2)))
    for p in Person for c1 in Cigar for c2 in Cigar
    if c1 != c2])
cnf.append(*[
    Clause(*[HasPet(p, pet) for pet in Pet])
    for p in Person])
cnf.append(*[
    Clause(Not(HasPet(p, pet1)), Not(HasPet(p, pet2)))
    for p in Person for pet1 in Pet for pet2 in Pet
    if pet1 != pet2])

# C-4. No owners have the same pet, smoke the same brand of cigar or drink the same beverage.
cnf.append(*[
    Clause(Not(HasPet(p1, pet)), Not(HasPet(p2, pet)))
    for p1 in Person for p2 in Person for pet in Pet
    if p1 != p2])
cnf.append(*[
    Clause(Not(Smokes(p1, c)), Not(Smokes(p2, c)))
    for p1 in Person for p2 in Person for c in Cigar
    if p1 != p2])
cnf.append(*[
    Clause(Not(Drinks(p1, b)), Not(Drinks(p2, b)))
    for p1 in Person for p2 in Person for b in Beverage
    if p1 != p2])

## -- Clues --
# 1. The Brit lives in the red house.
cnf.append(Clause(Lives(Person.BRIT, House.RED)))

# 2. The Swede keeps dogs.
cnf.append(Clause(HasPet(Person.SWEDE, Pet.DOGS)))

# 3. The Dane drinks tea.
cnf.append(Clause(Drinks(Person.DANE, Beverage.TEA)))

# 4. The green house is on the left of the white house.
cnf.append(Clause(Not(LocatesAt(House.WHITE, Location.FIRST))))
cnf.append(*[
    Clause(Not(LocatesAt(House.GREEN, l)), LocatesAt(House.WHITE, l + 1))
    for l in Location
    if l != Location.FIFTH])
cnf.append(Clause(Not(LocatesAt(House.GREEN, Location.FIFTH))))

# 5. The green house owner drinks coffee.
cnf.append(*[
    Clause(Not(Lives(p, House.GREEN)), Drinks(p, Beverage.COFFEE))
    for p in Person])

# 6. The person who smokes Pall Mall rears birds.
cnf.append(*[
    Clause(Not(Smokes(p, Cigar.PALL_MALL)), HasPet(p, Pet.BIRDS))
    for p in Person])

# 7. The owner of the yellow house smokes Dunhill.
cnf.append(*[
    Clause(Not(Lives(p, House.YELLOW)), Smokes(p, Cigar.DUNHILL))
    for p in Person])

# 8. The man living in the center house drinks milk.
cnf.append(*[
    Clause(Not(Lives(p, h)), Not(LocatesAt(h, Location.THIRD)), Drinks(p, Beverage.MILK))
    for p in Person for h in House])

# 9. The Norwegian lives in the first house.
cnf.append(*[
    Clause(Not(Lives(Person.NORWEGIAN, h)), LocatesAt(h, Location.FIRST))
    for h in House])

# 10. The man who smokes Blends lives next to the one who keeps cats.
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(HasPet(p2, Pet.CATS)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIRST)), LocatesAt(h2, Location.SECOND))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(HasPet(p2, Pet.CATS)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIFTH)), LocatesAt(h2, Location.FOURTH))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(HasPet(p2, Pet.CATS)), Not(Lives(p2, h2)), Not(LocatesAt(h1, l)), LocatesAt(h2, l + 1), LocatesAt(h2, l - 1))
    for p1 in Person for p2 in Person for h1 in House for h2 in House for l in Location
    if l != Location.FIRST and l != Location.FIFTH])

# 11. The man who keeps the horse lives next to the man who smokes Dunhill.
cnf.append(*[
    Clause(Not(HasPet(p1, Pet.HORSES)), Not(Lives(p1, h1)), Not(Smokes(p2, Cigar.DUNHILL)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIRST)), LocatesAt(h2, Location.SECOND))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(HasPet(p1, Pet.HORSES)), Not(Lives(p1, h1)), Not(Smokes(p2, Cigar.DUNHILL)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIFTH)), LocatesAt(h2, Location.FOURTH))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(HasPet(p1, Pet.HORSES)), Not(Lives(p1, h1)), Not(Smokes(p2, Cigar.DUNHILL)), Not(Lives(p2, h2)), Not(LocatesAt(h1, l)), LocatesAt(h2, l + 1), LocatesAt(h2, l - 1))
    for p1 in Person for p2 in Person for h1 in House for h2 in House for l in Location
    if l != Location.FIRST and l != Location.FIFTH])

# 12. The owner who smokes Blue Master drinks beer.
cnf.append(*[
    Clause(Not(Smokes(p, Cigar.BLUE_MASTER)), Drinks(p, Beverage.BEER))
    for p in Person])

# 13. The German smokes Prince.
cnf.append(Clause(Smokes(Person.GERMAN, Cigar.PRINCE)))

# 14. The Norwegian lives next to the blue house.
cnf.append(*[
    Clause(Not(Lives(Person.NORWEGIAN, h)), Not(LocatesAt(h, Location.FIRST)), LocatesAt(House.BLUE, Location.SECOND))
    for h in House])
cnf.append(*[
    Clause(Not(Lives(Person.NORWEGIAN, h)), Not(LocatesAt(h, Location.FIFTH)), LocatesAt(House.BLUE, Location.FOURTH))
    for h in House])
cnf.append(*[
    Clause(Not(Lives(Person.NORWEGIAN, h)), Not(LocatesAt(h, l)), LocatesAt(House.BLUE, l + 1), LocatesAt(House.BLUE, l - 1))
    for h in House for l in Location
    if l != Location.FIRST and l != Location.FIFTH])

# 15. The man who smokes Blends has a neighbor who drinks water.
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(Drinks(p2, Beverage.WATER)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIRST)), LocatesAt(h2, Location.SECOND))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(Drinks(p2, Beverage.WATER)), Not(Lives(p2, h2)), Not(LocatesAt(h1, Location.FIFTH)), LocatesAt(h2, Location.FOURTH))
    for p1 in Person for p2 in Person for h1 in House for h2 in House])
cnf.append(*[
    Clause(Not(Smokes(p1, Cigar.BLENDS)), Not(Lives(p1, h1)), Not(Drinks(p2, Beverage.WATER)), Not(Lives(p2, h2)), Not(LocatesAt(h1, l)), LocatesAt(h2, l + 1), LocatesAt(h2, l - 1))
    for p1 in Person for p2 in Person for h1 in House for h2 in House for l in Location
    if l != Location.FIRST and l != Location.FIFTH])

In [6]:
## Writing the CNF to a file

with open('puzzle.cnf', 'w') as f:
    f.write(f'p cnf {len(props)} {len(cnf)}\n')
    for clause in list(cnf):
        f.write(' '.join(map(str, clause)) + ' 0\n')

In [7]:
!clang++ dpll.cpp -o dpll -std=c++17 -O3
!./dpll puzzle.cnf puzzle.sol

In [8]:
# Parse the solution file

solve_status = {-1: 'UNKNOWN', 0: "UNSAT", 1: "SAT"}
solution = set()
model = []

with open('puzzle.sol', 'r') as f:
    for line in f:
        if line.startswith('t'):
            _, _, solved, _, _, etime, *_ = line.split()
            print(f'status: {solve_status[int(solved)]}, time: {etime} seconds')
        elif line.startswith('v'):
            var = int(line.split()[1])
            id, neg = abs(var), var < 0
            if not neg:
                solution.add(PropType.from_id(id))
            model.append(var)

if cnf.verify(model):
    print('Solution verified')
    locations = list(Location)
    houses = [h for l in locations for h in House if LocatesAt(h, l) in solution]
    persons = [p for h in houses for p in Person if Lives(p, h) in solution]
    beverages = [b for p in persons for b in Beverage if Drinks(p, b) in solution]
    cigars = [c for p in persons for c in Cigar if Smokes(p, c) in solution]
    pets = [pet for p in persons for pet in Pet if HasPet(p, pet) in solution]
    print('Location  ' + ''.join(format(l.name, '<12') for l in locations))
    print('House     ' + ''.join(format(h.name, '<12') for h in houses))
    print('Person    ' + ''.join(format(p.name, '<12') for p in persons))
    print('Beverage  ' + ''.join(format(b.name, '<12') for b in beverages))
    print('Cigar     ' + ''.join(format(c.name, '<12') for c in cigars))
    print('Pet       ' + ''.join(format(pet.name, '<12') for pet in pets))

status: SAT, time: 0.105 seconds


Solution verified
Location  FIRST       SECOND      THIRD       FOURTH      FIFTH       
House     YELLOW      BLUE        RED         GREEN       WHITE       
Person    NORWEGIAN   DANE        BRIT        GERMAN      SWEDE       
Beverage  WATER       TEA         MILK        COFFEE      BEER        
Cigar     DUNHILL     BLENDS      PALL_MALL   PRINCE      BLUE_MASTER 
Pet       CATS        HORSES      BIRDS       FISH        DOGS        
